In [ ]:
import numpy as np
from sklearn.metrics import mutual_info_score
from scipy.integrate import solve_ivp
import plotly.graph_objs as go
import pysindy as ps

# Parameters for the Lorenz system
sigma = 10.0
rho = 28.0
beta = 8.0 / 3.0

# Lorenz system equations
def lorenz(t, state):
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

# Initial conditions and integration time
initial_state = [1.0, 1.0, 1.0]
t_span = (0, 25)  # Time interval
t_eval = np.linspace(t_span[0], t_span[1], 1000)  # Time points

# Solve the Lorenz system
sol = solve_ivp(lorenz, t_span, initial_state, t_eval=t_eval)
x = sol.y[0]

# Function to compute mutual information
def mutual_information(x, lag):
    n = len(x)
    return mutual_info_score(x[:n - lag], x[lag:])

# Compute mutual information for different lags
lags = np.arange(1, 100)
mutual_infos = [mutual_information(x, lag) for lag in lags]

# Find the lag where mutual information reaches its first minimum
tau = lags[np.argmin(mutual_infos)]
print(f"Optimal tau: {tau}")

# Function to create delay vectors
def delay_embedding(data, delay, dim):
    n = len(data)
    embedded_data = np.zeros((n - (dim - 1) * delay, dim))
    for i in range(dim):
        embedded_data[:, i] = data[i * delay:n - (dim - 1 - i) * delay]
    return embedded_data

# Create delay vectors using the optimal tau
embedding_dim = 3
embedded_data = delay_embedding(x, tau, embedding_dim)

# Delay vectors for 2*tau
embedded_data_2tau = delay_embedding(x, 2 * tau, embedding_dim)

# Create an interactive Plotly graph
fig = go.Figure()

# Add graph for tau
fig.add_trace(go.Scatter3d(
    x=embedded_data[:, 0],
    y=embedded_data[:, 1],
    z=embedded_data[:, 2],
    mode='lines',
    name=f'Delay Embedding tau={tau}',
    line=dict(color='blue', width=2)
))

# Add graph for 2*tau
fig.add_trace(go.Scatter3d(
    x=embedded_data_2tau[:, 0],
    y=embedded_data_2tau[:, 1],
    z=embedded_data_2tau[:, 2],
    mode='lines',
    name=f'Delay Embedding tau={2 * tau}',
    line=dict(color='red', width=2)
))

# Configure the layout of the graph
fig.update_layout(
    title="Reconstructed Lorenz Attractor using Delay Embedding",
    scene=dict(
        xaxis_title='x(t)',
        yaxis_title=f'x(t-{tau})',
        zaxis_title=f'x(t-{2*tau})'
    ),
    width=800,
    height=600,
    margin=dict(l=0, r=0, b=0, t=40)
)

# Display the graph
fig.show()